Add a requirements file for all needed packages.(Place requirements.txt in the directory where you plan to run the command. If the file is in a different directory, you need to specify its path like path/to/requirements.txt )

In [ ]:
# %pip install -r requirements.txt
#%pip install \
#    presidio-analyzer \
#    presidio-anonymizer \
#    flair



In [ ]:
# model_name = "beki/flair-pii-distilbert"

In [ ]:
%pip install presidio-analyzer
%pip install presidio-anonymizer
%pip install flair
%pip install docx2txt
#%%python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 31.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 54.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 51.5 MB/s eta 0:0

In [ ]:
%%python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


2023-09-27 20:33:21.539278: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 20:33:24.822475: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import pandas as pd
import spacy as spacy
import numpy as np
import csv
import pprint
from typing import List,Optional,Dict, Union, Iterator, Iterable

from presidio_analyzer import AnalyzerEngine,BatchAnalyzerEngine, DictAnalyzerResult,RecognizerResult
from presidio_anonymizer import BatchAnonymizerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer import PatternRecognizer
from presidio_analyzer import Pattern


import collections
from dataclasses import dataclass
from presidio_anonymizer.entities import EngineResult

import os
import fnmatch
from pandas import read_excel

import docx2txt


Run Analysis on a sample *string*

In [ ]:
#from presidio_analyzer import AnalyzerEngine

# Set up the engine, loads the NLP module (spaCy model by default) and other PII recognizers
text= "My name is 212-555-5555 and I am a Hindu. you can reply to sam@yahoo.com. I would like to marry a Sunni.file under this 123-45-9754"
analyzer = AnalyzerEngine()

# Call analyzer to get results
analyzer_results = analyzer.analyze(text="My name is 212-555-5555 and I am a Hindu. you can reply to sam@yahoo.com. I would like to marry a Sunni."+
                                  "file under this 123-45-9754",
                           entities=["PHONE_NUMBER","NRP","EMAIL_ADDRESS","US_SSN"],
                           language='en')
print(analyzer_results)
pprint.pprint(analyzer_results)


[type: EMAIL_ADDRESS, start: 59, end: 72, score: 1.0, type: NRP, start: 35, end: 40, score: 0.85, type: US_SSN, start: 120, end: 131, score: 0.5, type: PHONE_NUMBER, start: 11, end: 23, score: 0.4]
[type: EMAIL_ADDRESS, start: 59, end: 72, score: 1.0,
 type: NRP, start: 35, end: 40, score: 0.85,
 type: US_SSN, start: 120, end: 131, score: 0.5,
 type: PHONE_NUMBER, start: 11, end: 23, score: 0.4]


In [ ]:
def generateCsvFromAnalyzerResults(analyzer_results,filename):

  view=[(text[res.start:res.end], res.start, res.end, res.entity_type) for res in analyzer_results]
  columns=['PII_Value','start_index','end_index','PII_Type']
  df_view=pd.DataFrame(view,columns=columns)
  nlp = spacy.load("en_core_web_sm")
  #print([sent.text for sent in nlp(text).sents])
  #format_numeric = lambda sent: f"{num:e}" if isinstance(num, int) else f"{num:,.2f}"
  #sentence = lambda sent: sent.text if sent.start_index<res.start & sent.end_index>res.end in nlp(text).sents
  #df_view['Sentence']=[sentence]
  generated_csvfile=df_view.to_csv(filename)
  return df_view

In [ ]:
csv=generateCsvFromAnalyzerResults(analyzer_results,'tokens_view2.csv')
csv

,PII_Value,start_index,end_index,PII_Type
0,sam@yahoo.com,59,72,EMAIL_ADDRESS
1,Hindu,35,40,NRP
2,123-45-9754,120,131,US_SSN
3,212-555-5555,11,23,PHONE_NUMBER


Batch Analyzer for CSV


In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
including classes from presidio that detects pii from dataframes and anonamizes them
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
@dataclass
class DictAnalyzerResult:
    """Hold the analyzer results per value or list of values."""
    key: str
    value: Union[str, List[str]]
    recognizer_results: Union[List[RecognizerResult], List[List[RecognizerResult]]]


class BatchAnalyzerEngine(AnalyzerEngine):
    """
    Class inheriting from AnalyzerEngine and adds the funtionality to analyze lists or dictionaries.
    """

    def analyze_list(self, list_of_texts: Iterable[str], **kwargs) -> List[List[RecognizerResult]]:
        """
        Analyze an iterable of strings

        :param list_of_texts: An iterable containing strings to be analyzed.
        :param kwargs: Additional parameters for the `AnalyzerEngine.analyze` method.
        """

        list_results = []
        for text in list_of_texts:
            results = self.analyze(text=text, **kwargs) if isinstance(text, str) else []
            list_results.append(results)
        return list_results

    def analyze_dict(
     self, input_dict: Dict[str, Union[object, Iterable[object]]], **kwargs) -> Iterator[DictAnalyzerResult]:
        """
        Analyze a dictionary of keys (strings) and values (either object or Iterable[object]).
        Non-string values are returned as is.

                :param input_dict: The input dictionary for analysis
        :param kwargs: Additional keyword arguments for the `AnalyzerEngine.analyze` method
        """

        for key, value in input_dict.items():
            if not value:
                results = []
            else:
                if isinstance(value, str):
                    results: List[RecognizerResult] = self.analyze(text=value, **kwargs)
                elif isinstance(value, collections.abc.Iterable):
                    results: List[List[RecognizerResult]] = self.analyze_list(
                                list_of_texts=value,
                                **kwargs)
                else:
                    results = []
            yield DictAnalyzerResult(key=key, value=value, recognizer_results=results)

if __name__ == '__main__':

    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
        updating presidio by including different regular expressions and lists
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

    batch_analyzer = BatchAnalyzerEngine()
    analyzer = AnalyzerEngine()



    # Adding zip code in the entity list. Make sure zip code is turned into a string for regex to work
    zip_pattern = Pattern(name="zip_pattern",regex= '(\\b\\d{5}(?:\\-\\d{4})?\\b)', score = 0.5)#regular expression that selects zip codes
    zip_recognizer = PatternRecognizer(supported_entity="ZIPCODE", #name of new entity
                                    patterns = [zip_pattern],
                                    context= ["zip","zipcode"])#including any surrounding words that has zip or zipcode in it
    batch_analyzer.registry.add_recognizer(zip_recognizer)#adding new zip code recognizer to the model
    analyzer.registry.add_recognizer(zip_recognizer)


    #Adding State
    # state_recognizer = PatternRecognizer(supported_entity="STATE",#name of new entity
    #                                     deny_list=list(location_list['State short'].dropna().unique()),#only include unique states, drop null values
    #                                     context= ["state","address"])#including any surrounding words that has state or address in it
    # batch_analyzer.registry.add_recognizer(state_recognizer)#adding new state recognizer to the model
    # analyzer.registry.add_recognizer(state_recognizer)

    # #Adding List of Cities
    # city_recognizer = PatternRecognizer(supported_entity="CITY",#name of new entity
    #                                   deny_list=list(location_list['City'].dropna().unique()),#only include unique city, drop null values
    #                                   context= ["city","address"])#including any surrounding words that has city or address in it

    # batch_analyzer.registry.add_recognizer(city_recognizer)#adding new city recognizer to the model
    # analyzer.registry.add_recognizer(city_recognizer)

    #Adding Password


    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    Running presidio on all types of files
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

    for filename in os.listdir('.'):
        if fnmatch.fnmatch(filename, 's_pii_*.txt'):

            df = pd.read_csv(filename, index_col = 0).reset_index(drop = True)
            df = df.astype(str)
            #df['zip'] = df['zip'].astype(str)
            #df['phone numbers'] = df['phone numbers'].astype(str)
            df_dict = df.to_dict(orient="list") #df being converted to a dictionary
            analyzer_results = batch_analyzer.analyze_dict(df_dict, language="en")
            analyzer_df = pd.DataFrame(analyzer_results) #converting into a dataframe
            presidio_df = pd.DataFrame(list(analyzer_df['recognizer_results']), analyzer_df['key']).reset_index()
            presidio_df.insert(0, 'filename', filename, True)


    for filename in os.listdir('.'):
        if fnmatch.fnmatch(filename, 'SensitiveData*.csv'):
            #from csv files
            csv = pd.read_csv(filename)
            csv = csv.astype(str).replace('nan',np.nan)
            df_dict_csv = csv.to_dict(orient="list") #df being converted to a dictionary
            analyzer_results_csv = batch_analyzer.analyze_dict(df_dict_csv, language="en")
            csv=generateCsvFromAnalyzerResults(analyzer_results,'tokens_view2.csv')
            analyzer_df_csv = pd.DataFrame(analyzer_results_csv) #converting into a dataframe
            presidio_df_csv = pd.DataFrame(list(analyzer_df_csv['recognizer_results']), analyzer_df_csv['key']).reset_index()
            presidio_df_csv.insert(0, 'filename', filename, True)


    for filename in os.listdir('.'):
        if fnmatch.fnmatch(filename, 's_pii_*.xlsx'):
            xlsx = pd.read_excel(filename, engine = 'openpyxl')
            xlsx = xlsx.astype(str).replace('nan',np.nan)
            df_dict_xlsx = xlsx.to_dict(orient="list") #df being converted to a dictionary
            analyzer_results_xlsx = batch_analyzer.analyze_dict(df_dict_xlsx, language="en")
            analyzer_df_xlsx = pd.DataFrame(analyzer_results_xlsx) #converting into a dataframe
            presidio_df_xlsx = pd.DataFrame(list(analyzer_df_xlsx['recognizer_results']), analyzer_df_xlsx['key']).reset_index()
            presidio_df_xlsx.insert(0, 'filename', filename, True)


    for filename in os.listdir('.'):
        if fnmatch.fnmatch(filename, 's_pii_*.docx'):
            MY_TEXT = docx2txt.process(filename)
            with open("pii_docx_made.txt", "w") as text_file:
                print(MY_TEXT, file=text_file)

            docx = pd.read_csv("pii_docx_made.txt", sep = "\t")
            df_dict_docx = docx.to_dict(orient="list") #df being converted to a dictionary
            analyzer_results_docx = batch_analyzer.analyze_dict(df_dict_docx, language="en")
            analyzer_df_docx = pd.DataFrame(analyzer_results_docx) #converting into a dataframe
            presidio_df_docx = pd.DataFrame(list(analyzer_df_docx['recognizer_results']), analyzer_df_docx['key']).reset_index()
            presidio_df_docx.insert(0, 'filename', filename, True)



    #frames = [presidio_df, presidio_df_csv, presidio_df_xlsx, presidio_df_docx]
    frames = [presidio_df,presidio_df_csv]
    final = pd.concat(frames)
    final.to_csv("result_structured.csv")

In [ ]:
@dataclass
class DictAnalyzerResult:
    """Hold the analyzer results per value or list of values."""
    key: str
    value: Union[str, List[str]]
    recognizer_results: Union[List[RecognizerResult], List[List[RecognizerResult]]]


class BatchAnalyzerEngine(AnalyzerEngine):
    """
    Class inheriting from AnalyzerEngine and adds the funtionality to analyze lists or dictionaries.
    """

    def analyze_list(self, list_of_texts: Iterable[str], **kwargs) -> List[List[RecognizerResult]]:
        """
        Analyze an iterable of strings

        :param list_of_texts: An iterable containing strings to be analyzed.
        :param kwargs: Additional parameters for the `AnalyzerEngine.analyze` method.
        """

        list_results = []
        for text in list_of_texts:
            results = self.analyze(text=text, **kwargs) if isinstance(text, str) else []
            list_results.append(results)
        return list_results

    def analyze_dict(
     self, input_dict: Dict[str, Union[object, Iterable[object]]], **kwargs) -> Iterator[DictAnalyzerResult]:
        """
        Analyze a dictionary of keys (strings) and values (either object or Iterable[object]).
        Non-string values are returned as is.

                :param input_dict: The input dictionary for analysis
        :param kwargs: Additional keyword arguments for the `AnalyzerEngine.analyze` method
        """

        for key, value in input_dict.items():
            if not value:
                results = []
            else:
                if isinstance(value, str):
                    results: List[RecognizerResult] = self.analyze(text=value, **kwargs)
                elif isinstance(value, collections.abc.Iterable):
                    results: List[List[RecognizerResult]] = self.analyze_list(
                                list_of_texts=value,
                                **kwargs)
                else:
                    results = []
            yield DictAnalyzerResult(key=key, value=value, recognizer_results=results)

if __name__ == '__main__':

    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
        updating presidio by including different regular expressions and lists
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

    batch_analyzer = BatchAnalyzerEngine()
    analyzer = AnalyzerEngine()




CSV FILE

In [ ]:
filename='/content/SensitiveData.csv'
csv = pd.read_csv(filename)
csv = csv.astype(str).replace('nan',np.nan)
df_dict_csv = csv.to_dict(orient="list") #df being converted to a dictionary


analyzer_results = batch_analyzer.analyze_dict(df_dict_csv, language="en")
analyzer_results = list(analyzer_results)
analyzer_results
#pprint.pprint(analyzer_results)
pd.DataFrame(analyzer_results).to_csv('Sensitivedata_PII_Results.csv')


Anonymizer

In [ ]:
batch_anonymizer = BatchAnonymizerEngine()
anonymizer_results = batch_anonymizer.anonymize_dict(analyzer_results)

scrubbed_df = pd.DataFrame(anonymizer_results)
scrubbed_df


,text
0,My lucky number is <US_SSN>
1,"""<PERSON> Social Security Number is <US_SSN>."""
2,"""<DATE_TIME> is <PERSON>'s birthday, and he tu..."
3,"""Please find your account number: <US_BANK_NUM..."
4,"""This is a random text without sensitive data."""
5,"""<PERSON> works at XYZ Corp."""
6,"""Her email is <EMAIL_ADDRESS>."""
7,"""Khyat Doe's Social Security Number is <US_SSN>."""
8,"""Xs Social Security Number is <US_SSN>."""
9,"""Y Social Security Number is <DATE_TIME>."""


TXT FILE

In [ ]:
filename='/content/s_pii_txt.txt'
df = pd.read_csv(filename, index_col = 0).reset_index(drop = True)
df = df.astype(str)
            #df['zip'] = df['zip'].astype(str)
            #df['phone numbers'] = df['phone numbers'].astype(str)
df_dict = df.to_dict(orient="list") #df being converted to a dictionary
analyzer_results = batch_analyzer.analyze_dict(df_dict, language="en")
analyzer_df = pd.DataFrame(analyzer_results) #converting into a dataframe
#presidio_df = pd.DataFrame(list(analyzer_df['recognizer_results']), analyzer_df['key']).reset_index()
analyzer_df.to_csv('text_pii_result.csv')
#pprint.pprint(analyzer_df)

SAMPLE TEST CSV file

In [ ]:
filename='/content/sample-data.csv'
csv = pd.read_csv(filename)
csv = csv.astype(str).replace('nan',np.nan)
df_dict_csv = csv.to_dict(orient="list") #df being converted to a dictionary


analyzer_results = batch_analyzer.analyze_dict(df_dict_csv, language="en")
analyzer_results = list(analyzer_results)
analyzer_results
pd.DataFrame(analyzer_results).to_csv('sample-data_pii_result.csv')
#pprint.pprint(analyzer_results)